# Introduction

This work is based on the implementation of the paper [*Personalized Federated Learning with Moreau Envelopes*](https://arxiv.org/pdf/2006.08848). The implementation can be found [here](https://github.com/CharlieDinh/pFedMe). Note that this repository not only implements pFedMe but also FedAvg and Per-FedAvg algorithms that will allow further analysis. 

In this project, we focus exclusively on the MNIST dataset. We investigate the impact of using different levels of non-iidness and the effects of data poisoning attacks.

In [1]:
from utils.plot_result_utils import *

# Baseline

In order to establish a baseline for comparison and to replicate the results of the paper, we executed the following commands:

- **Strongly Convex Case:**
    - pFedMe:
        ```
        python3 main.py --dataset Mnist --model mclr --batch_size 20 --learning_rate 0.01 --personal_learning_rate 0.1 --beta 2 --lamda 15 --num_global_iters 800 --local_epochs 20 --algorithm pFedMe --numusers 5 --times 10
        ```
    - FedAvg:
        ```
        python3 main.py --dataset Mnist --model mclr --batch_size 20 --learning_rate 0.02 --num_global_iters 800 --local_epochs 20 --algorithm FedAvg --numusers 5 --times 10
        ```
    - PerAvg:
        ```
        python3 main.py --dataset Mnist --model mclr --batch_size 20 --learning_rate 0.03 --beta 0.003  --num_global_iters 800 --local_epochs 20 --algorithm PerAvg --numusers 5 --times 10
        ```

- **Non-Convex Case:**
    - pFedMe:
        ```
        python3 main.py --dataset Mnist --model dnn --batch_size 20 --learning_rate 0.01 --personal_learning_rate 0.05 --beta 2 --lamda 30 --num_global_iters 800 --local_epochs 20 --algorithm pFedMe --numusers 5 --times 10
        ```
    - FedAvg:
        ```
        python3 main.py --dataset Mnist --model dnn --batch_size 20 --learning_rate 0.02 --num_global_iters 800 --local_epochs 20 --algorithm FedAvg --numusers 5 --times 10
        ```
    - PerAvg:
        ```
        python3 main.py --dataset Mnist --model dnn --batch_size 20 --learning_rate 0.02 --beta 0.001  --num_global_iters 800 --local_epochs 20 --algorithm PerAvg --numusers 5 --times 10
        ```

All results are stored in the 'results_baseline' folder.

Each algorithm is executed at least 10 times, and the results are then averaged using the `average_data()` function in the [plot_utils.py](./utils/plot_utils.py) file. The averaged results are stored in the XXX_avg.h5 files. Additionally, the results of each training round are stored in the XXX.h5 files using the `save_results()` function in the [serverbase.py](./FLAlgorithms/servers/serverbase.py) file.

In [2]:
num_users = 5
folders = ["./results_baseline/results_DNN", "./results_baseline/results_MLR"]

## Average

In [3]:
max_average_df(num_users=num_users, folders=folders)

,Algorithm,Folder,Max testing Accuracy,Index
0,pFedMe_p,results_DNN,0.967495,255
1,pFedMe,results_DNN,0.962392,768
2,PerAvg_p,results_DNN,0.941550,794
3,FedAvg,results_DNN,0.960448,570
4,pFedMe_p,results_MLR,0.939390,76
5,pFedMe,results_MLR,0.919438,250
6,PerAvg_p,results_MLR,0.933477,470
7,FedAvg,results_MLR,0.925000,743


## Training rounds

In [4]:
max_df(num_users=num_users, folders=folders)

,Algorithm,Folder,Max testing Accuracy,Index,Mean Accuracy,Variance
0,FedAvg,results_DNN,0.962743,556,0.961744,2.437710e-07
1,FedAvg,results_MLR,0.929266,780,0.928483,4.284214e-07
2,PerAvg_p,results_DNN,0.942765,795,0.941712,6.551852e-07
3,PerAvg_p,results_MLR,0.935745,470,0.934746,5.353243e-07
4,pFedMe,results_DNN,0.965173,782,0.963688,4.413794e-07
5,pFedMe,results_MLR,0.924406,433,0.923785,3.733503e-07
6,pFedMe_p,results_DNN,0.970572,184,0.969546,5.636698e-07
7,pFedMe_p,results_MLR,0.944114,39,0.942414,1.620551e-06


# Non-iidness

# Attacks

In [5]:
num_users = 5
folders = ["./results_attacks/results_DNN", "./results_attacks/results_MLR"]

## Average

In [6]:
max_average_df(num_users=num_users, folders=folders)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './results_attacks/results_DNN\Mnist_pFedMe_p_0.01_2.0_30_5u_20b_20_5_0.05_avg.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Training rounds

In [ ]:
max_df(num_users=num_users, folders=folders)